Sweet Success: Analyzing Recipe Ratings with Data Science

**Name(s)**: Timothy Kam

**Website Link**: https://psionergy.github.io/recipe-analysis-project/

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path

import plotly.express as px
pd.options.plotting.backend = 'plotly'

#from dsc80_utils import * # Feel free to uncomment and use this.

recipes = pd.read_csv('food_data/RAW_recipes.csv')
ratings = pd.read_csv('food_data/RAW_interactions.csv')
ratings_dist = ratings['rating'].value_counts().sort_index()

# Generate a bar plot of ratings distribution
fig = px.bar(
    x=ratings_dist.index,
    y=ratings_dist.values,
    labels={'x': 'Rating', 'y': 'Count'},
    title='Distribution of Ratings'
)

fig.show()

## Step 1: Introduction

In [ ]:
# # Step 1: Introduction and Question Identification

## Dataset Introduction
## The dataset I am working with is the **Recipes and Ratings** dataset. It contains information about recipes, their preparation times, ingredients, user ratings, and reviews.

## Central Question
## How do the number of ingredients and preparation time affect recipe ratings?*

## Why It Matters
## Understanding these relationships can help recipe creators optimize their content for better user satisfaction. It also sheds light on user behavior and preferences in cooking.

### Relevant Columns
## - `rating`: User rating of the recipe (target variable).
## - `n_ingredients`: Number of ingredients in a recipe.
## - `minutes`: Preparation time for the recipe.

### Dataset Overview
## ```python
# Display basic information about the dataset
print(f"Recipes Dataset: {recipes.shape[0]} rows, {recipes.shape[1]} columns")
print(f"Ratings Dataset: {ratings.shape[0]} rows, {ratings.shape[1]} columns")

# Show the first few rows
print("Recipes Dataset Head:")
display(recipes.head())

print("\nRatings Dataset Head:")
display(ratings.head())


## Step 2: Data Cleaning and Exploratory Data Analysis

In [ ]:
### Data Cleaning
### The following data cleaning steps were performed to ensure the dataset was ready for analysis:

### 1. **Handled Missing Values**:
###   - Replaced invalid values (`'?'`, `'NA'`, `'null'`) with `NaN`.
###   - Converted the `date` column in the ratings dataset to a datetime object.

### 2. **Ensured Numeric Columns**:
###  - Ensured columns such as `n_ingredients` and `rating` were numeric for accurate analysis.

### 3. **Removed Duplicates**:
##   - Removed duplicate rows in both the `recipes` and `ratings` datasets.

### These cleaning steps ensured that the data was consistent and ready for further analysis. 
###
# # Replace invalid values with NaN
recipes.replace(['?', 'NA', 'null'], np.nan, inplace=True)
ratings.replace(['?', 'NA', 'null'], np.nan, inplace=True)

# Convert 'date' column in ratings to datetime
ratings['date'] = pd.to_datetime(ratings['date'])

# Drop duplicates
recipes = recipes.drop_duplicates()
ratings = ratings.drop_duplicates()

# Ensure numeric columns
recipes['n_ingredients'] = pd.to_numeric(recipes['n_ingredients'], errors='coerce')
ratings['rating'] = pd.to_numeric(ratings['rating'], errors='coerce')

# Display cleaned data
print("Cleaned Recipes Dataset:")
display(recipes.head())


In [ ]:
### Univariate Analysis

##1. **Distribution of Ratings**:
 ##  - Most recipes receive ratings of 4 or 5, indicating a positive user bias.

##2. **Distribution of Preparation Times**:
 ##  - Majority of recipes take under 60 minutes to prepare.


# Plot 1: Distribution of Ratings
##ratings_dist = ratings['rating'].value_counts().sort_index()
fig1 = px.bar(
    x=ratings_dist.index,
    y=ratings_dist.values,
    labels={'x': 'Rating', 'y': 'Count'},
    title='Distribution of Ratings'
)
fig1.show()




# Filter the data to include only preparation times within the 0–300 range
filtered_recipes = recipes[recipes['minutes'] <= 300]

# Plot the histogram for preparation times in the filtered range
fig = px.histogram(
    filtered_recipes,
    x='minutes',
    nbins=50,
    title='Distribution of Recipe Preparation Times (0-300 Minutes)',
    labels={'minutes': 'Preparation Time (Minutes)', 'count': 'Recipe Count'}
)

# Update the layout with explicit axis labels and formatting
fig.update_layout(
    xaxis_title="Preparation Time (Minutes)",
    yaxis_title="Number of Recipes",
    xaxis=dict(range=[0, 300])  # Set the range explicitly
)

fig.show()
# Save the first graph (ratings distribution) to an HTML file
fig1.write_html('assets/ratings_distribution.html', include_plotlyjs='cdn')

# Save the second graph (preparation times) to an HTML file
fig.write_html('assets/preparation_times.html', include_plotlyjs='cdn')


In [ ]:
### Bivariate Analysis

##1. **Number of Ingredients vs. Preparation Time**:
##   - Recipes with more ingredients tend to take longer to prepare.

##2. **Ratings by Year**:
##   - Ratings have been consistently high over the years.


# Plot 1: Ingredients vs. Preparation Time
# Plot 1: Ingredients vs. Preparation Time
fig3 = px.scatter(
    recipes,
    x='n_ingredients',
    y='minutes',
    title='Ingredients vs. Preparation Time',
    labels={'x': 'Number of Ingredients', 'y': 'Preparation Time (minutes)'}
)
fig3.show()

# Ensure 'date' column is in datetime format
ratings['date'] = pd.to_datetime(ratings['date'], errors='coerce')

# Extract the year from the 'date' column
ratings['year'] = ratings['date'].dt.year

# Plot 2: Ratings by Year
fig4 = px.box(
    ratings,
    x='year',
    y='rating',
    title='Ratings by Year',
    labels={'x': 'Year', 'y': 'Rating'}
)
fig4.show()



In [ ]:
# Manually create a Markdown table
def generate_markdown_table(df, num_rows=10):
    # Get the first few rows of the DataFrame
    rows = df.head(num_rows).values.tolist()
    # Get the column headers
    headers = df.columns.tolist()
    
    # Start with the header row
    markdown = "| " + " | ".join(headers) + " |\n"
    markdown += "| " + " | ".join(["---"] * len(headers)) + " |\n"
    
    # Add each row of data
    for row in rows:
        markdown += "| " + " | ".join(map(str, row)) + " |\n"
    
    return markdown

# Generate the Markdown table for the interesting aggregates
markdown_table = generate_markdown_table(interesting_aggregates)

# Print the Markdown table
print(markdown_table)


## Step 3: Assessment of Missingness

In [ ]:
## Step 3: Assessment of Missingness

### NMAR Analysis
##The column `review` may be NMAR since missing reviews could depend on user behavior (e.g., users who dislike a recipe may skip leaving a review).

### Missingness Dependency
##Test whether missingness in `review` is dependent on `rating`.


# Test for dependency
missing_reviews = ratings['review'].isnull()
obs_diff = ratings.groupby(missing_reviews)['rating'].mean()

# Permutation Test
np.random.seed(0)
perm_diffs = []
for _ in range(1000):
    shuffled = ratings['rating'].sample(frac=1).reset_index(drop=True)
    perm_diff = shuffled[missing_reviews].mean() - shuffled[~missing_reviews].mean()
    perm_diffs.append(perm_diff)

# Plot Permutation Results
fig = px.histogram(
    perm_diffs,
    title='Permutation Test: Missingness in Reviews vs Ratings',
    labels={'x': 'Difference in Means'}
)
fig.show()


## Step 4: Hypothesis Testing

In [ ]:
## Step 4: Hypothesis Testing

## **New Hypotheses**:
# - Null: Recipes with high sugar content (above the median) have the same ratings as those with low sugar content.
# - Alternative: Recipes with high sugar content have different ratings than those with low sugar content.

# Extract sugar content from the 'nutrition' column (assuming it's in a list or string format)
# If 'nutrition' is a string like "[calories, sugar, fat, ...]", extract the sugar column
recipes['sugar'] = recipes['nutrition'].apply(lambda x: float(x.strip('[]').split(',')[1]) if pd.notnull(x) else None)

# Merge the DataFrames on recipe_id
merged_df = pd.merge(ratings, recipes, left_on='recipe_id', right_on='id', how='inner')

# Categorize recipes into low and high sugar groups based on the median
sugar_median = merged_df['sugar'].median()
low_sugar = merged_df[merged_df['sugar'] <= sugar_median]['rating']
high_sugar = merged_df[merged_df['sugar'] > sugar_median]['rating']

# Check descriptive statistics for both groups
print(f"Low Sugar Ratings: {low_sugar.describe()}")
print(f"High Sugar Ratings: {high_sugar.describe()}")

# Perform the T-Test
from scipy.stats import ttest_ind
t_stat, p_val = ttest_ind(low_sugar, high_sugar, nan_policy='omit')
print(f"T-statistic: {t_stat}, P-value: {p_val}")

mean_diff = low_sugar.mean() - high_sugar.mean()
pooled_sd = np.sqrt((low_sugar.var() + high_sugar.var()) / 2)
cohens_d = mean_diff / pooled_sd
print(f"Cohen's d: {cohens_d}")

# Categorize recipes for visualization
merged_df['sugar_category'] = merged_df['sugar'].apply(lambda x: 'Low Sugar' if x <= sugar_median else 'High Sugar')

# Create a boxplot
import plotly.express as px
fig = px.box(
    merged_df,
    x='sugar_category',
    y='rating',
    title='Ratings by Sugar Content',
    labels={'sugar_category': 'Sugar Content Category', 'rating': 'Rating'}
)
fig.show()


## Step 5: Framing a Prediction Problem

In [ ]:
# # Step 5: Framing a Prediction Problem

# ## Prediction Problem
# I will predict recipe preparation time (in minutes) based on recipe characteristics.

# ## Type of Prediction
# This is a regression problem since we are predicting a continuous numerical value (minutes of preparation time).

# ## Response Variable
# The response variable is 'minutes' (preparation time) from the recipes dataset. I chose this because:
# 1. It's an objective measure that's directly influenced by recipe features
# 2. Has clear cause-and-effect relationships with characteristics like number of ingredients and steps
# 3. Provides practical value to users planning their cooking time

# ## Features Available at Prediction Time
# We can use recipe characteristics that would be known before cooking:
# - n_ingredients: Number of ingredients
# - n_steps: Number of steps
# - nutrition: Nutritional information
# - Description and tag information

# We cannot use features that would only be known after cooking, such as:
# - User ratings
# - Reviews
# - User interactions

# ## Evaluation Metric
# I will use Root Mean Squared Error (RMSE) to evaluate the model because:
# 1. It's appropriate for regression problems
# 2. Provides error in the same units as our prediction (minutes)
# 3. Penalizes larger errors more heavily, which is important for time predictions
# 4. Easy to interpret - tells us how far off our time predictions are on average

## Initial Investigation
# Let's look at some basic statistics about preparation time:

# Our target variable statistics show:
# 1. Preparation times range from 0 to 1,051,200 minutes (about 2 years)
# 2. Most recipes take between 20-65 minutes (25th to 75th percentile)
# 3. The median preparation time is 35 minutes
# 4. There are clear outliers in the data, as indicated by:
#    - Very high maximum value (over 2 years)
#    - Large difference between mean (115 minutes) and median (35 minutes)
#    - Large standard deviation (3,991 minutes)

# The correlations between preparation time and recipe features show:
# - n_ingredients correlation: -0.008
# - n_steps correlation: 0.008

# These very low correlations suggest we'll need to:
# 1. Clean outliers from our preparation times
# 2. Consider non-linear relationships
# 3. Engineer features that better capture recipe complexity

# This investigation will help inform our modeling choices in the subsequent steps.
# Basic statistics of our target variable
print("\nPreparation Time Statistics (minutes):")
print(recipes['minutes'].describe())

# Look at correlations with key features
correlations = recipes[['minutes', 'n_ingredients', 'n_steps']].corr()['minutes']
print("\nCorrelations with preparation time:")
print(correlations)

## Step 6: Baseline Model

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd
import numpy as np

# Step 1: Ensure all necessary features are computed
recipes['mean_ingredient_time'] = recipes['minutes'] / recipes['n_ingredients']
recipes['ingredients_per_step'] = recipes['n_ingredients'] / recipes['n_steps']
recipes['log_minutes'] = np.log1p(recipes['minutes'])

# Step 2: Define features and target
features = ['n_ingredients', 'n_steps', 'mean_ingredient_time', 'ingredients_per_step']
target = 'log_minutes'

# Step 3: Split dataset into training and testing sets
X = recipes[features]
y = recipes[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 4: Create pipeline with polynomial features
baseline_pipeline = Pipeline([
    ('scaler', StandardScaler()),  # Standardize features
    ('poly', PolynomialFeatures(degree=2, interaction_only=False, include_bias=False)),  # Add polynomial features
    ('regressor', Ridge())  # Ridge regression
])

# Step 5: Hyperparameter tuning
param_grid = {'regressor__alpha': [0.01, 0.1, 1, 10, 100]}  # Search for the best alpha (regularization strength)
grid_search = GridSearchCV(baseline_pipeline, param_grid, cv=3, scoring='neg_mean_squared_error', verbose=1)
grid_search.fit(X_train, y_train)

# Step 6: Evaluate the model
best_model = grid_search.best_estimator_
y_train_pred = best_model.predict(X_train)
y_test_pred = best_model.predict(X_test)

train_mse = mean_squared_error(y_train, y_train_pred)
test_mse = mean_squared_error(y_test, y_test_pred)
train_r2 = r2_score(y_train, y_train_pred)
test_r2 = r2_score(y_test, y_test_pred)

print("Improved Baseline Model Performance (Ridge Regression with Polynomial Features):")
print(f"Training MSE: {train_mse:.4f}")
print(f"Testing MSE: {test_mse:.4f}")
print(f"Training R^2: {train_r2:.4f}")
print(f"Testing R^2: {test_r2:.4f}")

# Step 7: Display feature coefficients
coefficients = best_model.named_steps['regressor'].coef_
poly_features = best_model.named_steps['poly'].get_feature_names_out(features)
feature_importance = pd.DataFrame({
    'Feature': poly_features,
    'Coefficient': coefficients
}).sort_values(by='Coefficient', ascending=False)

print("\nFeature Coefficients:")
print(feature_importance)

# Step 8: Best hyperparameters
print("\nBest Hyperparameters:")
print(grid_search.best_params_)


## Step 7: Final Model

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd

# Define features and target
features = ['n_ingredients', 'n_steps', 'mean_ingredient_time', 'ingredients_per_step']
target = 'log_minutes'

# Ensure the dataset is split correctly
X = recipes[features]
y = recipes[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 1: Create the pipeline
final_pipeline = Pipeline([
    ('scaler', StandardScaler()),  # Standardize features
    ('poly', PolynomialFeatures(degree=2, include_bias=False)),  # Polynomial features
    ('regressor', RandomForestRegressor(random_state=42))  # Random Forest for better performance
])

# Step 2: Hyperparameter tuning
param_grid = {
    'regressor__n_estimators': [50, 100],
    'regressor__max_depth': [5, 10],
    'regressor__min_samples_split': [2, 5],
}

grid_search = GridSearchCV(
    final_pipeline, 
    param_grid, 
    cv=3, 
    scoring='neg_mean_squared_error', 
    verbose=1
)
grid_search.fit(X_train, y_train)

# Step 3: Evaluate the best model
best_model = grid_search.best_estimator_
y_train_pred = best_model.predict(X_train)
y_test_pred = best_model.predict(X_test)

train_mse = mean_squared_error(y_train, y_train_pred)
test_mse = mean_squared_error(y_test, y_test_pred)
train_r2 = r2_score(y_train, y_train_pred)
test_r2 = r2_score(y_test, y_test_pred)

print("Final Model Performance:")
print(f"Training MSE: {train_mse:.4f}")
print(f"Testing MSE: {test_mse:.4f}")
print(f"Training R^2: {train_r2:.4f}")
print(f"Testing R^2: {test_r2:.4f}")

# Step 4: Feature importance
# Extract feature names from PolynomialFeatures
polynomial_transformer = best_model.named_steps['poly']
poly_features = polynomial_transformer.get_feature_names_out(features)

# Extract feature importances from Random Forest
importances = best_model.named_steps['regressor'].feature_importances_

# Align the lengths of feature names and importances
if len(poly_features) != len(importances):
    poly_features = poly_features[:len(importances)]

# Create a DataFrame to display feature importance
feature_importance = pd.DataFrame({
    'Feature': poly_features,
    'Importance': importances
}).sort_values(by='Importance', ascending=False)

print("\nFeature Importances:")
print(feature_importance)

# Step 5: Best hyperparameters
print("\nBest Hyperparameters:")
print(grid_search.best_params_)


## Step 8: Fairness Analysis

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error

# Step 1: Define groups
median_ingredients = X_test['n_ingredients'].median()
X_test['complexity_group'] = X_test['n_ingredients'].apply(
    lambda x: 'Low Complexity' if x <= median_ingredients else 'High Complexity'
)

# Step 2: Predict using the final model
y_test_pred = best_model.predict(X_test.drop(columns=['complexity_group']))

# Step 3: Calculate RMSE for each group
low_complexity_rmse = np.sqrt(mean_squared_error(
    y_test[X_test['complexity_group'] == 'Low Complexity'], 
    y_test_pred[X_test['complexity_group'] == 'Low Complexity']
))
high_complexity_rmse = np.sqrt(mean_squared_error(
    y_test[X_test['complexity_group'] == 'High Complexity'], 
    y_test_pred[X_test['complexity_group'] == 'High Complexity']
))

# Observed difference in RMSE
observed_diff = np.abs(low_complexity_rmse - high_complexity_rmse)
print(f"Observed RMSE Difference: {observed_diff:.4f}")

# Step 4: Permutation test
n_permutations = 1000
perm_diffs = []

# Combine true values and predictions
test_data = pd.DataFrame({
    'true': y_test,
    'predicted': y_test_pred,
    'complexity_group': X_test['complexity_group']
})

for _ in range(n_permutations):
    # Shuffle group labels
    shuffled = test_data.copy()
    shuffled['complexity_group'] = np.random.permutation(shuffled['complexity_group'])

    # Recalculate RMSE for permuted groups
    low_complexity_rmse_perm = np.sqrt(mean_squared_error(
        shuffled[shuffled['complexity_group'] == 'Low Complexity']['true'],
        shuffled[shuffled['complexity_group'] == 'Low Complexity']['predicted']
    ))
    high_complexity_rmse_perm = np.sqrt(mean_squared_error(
        shuffled[shuffled['complexity_group'] == 'High Complexity']['true'],
        shuffled[shuffled['complexity_group'] == 'High Complexity']['predicted']
    ))

    # Store permuted RMSE difference
    perm_diffs.append(np.abs(low_complexity_rmse_perm - high_complexity_rmse_perm))

# Step 5: Calculate p-value
perm_diffs = np.array(perm_diffs)
p_value = np.mean(perm_diffs >= observed_diff)
print(f"P-value: {p_value:.4f}")

# Step 6: Visualize the null distribution
import matplotlib.pyplot as plt

plt.hist(perm_diffs, bins=30, alpha=0.75, label='Null Distribution')
plt.axvline(observed_diff, color='red', linestyle='dashed', linewidth=2, label='Observed Difference')
plt.title('Permutation Test: RMSE Difference Between Groups')
plt.xlabel('RMSE Difference')
plt.ylabel('Frequency')
plt.legend()
plt.show()

# Step 7: Conclusion
if p_value < 0.05:
    print("Reject the null hypothesis: The model's performance differs significantly between the two groups.")
else:
    print("Fail to reject the null hypothesis: No significant difference in model performance between the two groups.")
